In [1]:
import pandas as pd
import numpy as np
import torch

In [ ]:
data_path = "C:/Users/minkyu/Desktop/open/"
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'

In [3]:
train = pd.read_csv(data_path+"train.csv" )

In [8]:
train.head(1)

,ID,발생일시,사고인지 시간,날씨,기온,습도,공사종류,연면적,층 정보,인적사고,물적사고,공종,사고객체,작업프로세스,장소,부위,사고원인,재발방지대책 및 향후조치계획
0,TRAIN_00000,2023-12-31 오후 12:44,정규작업 -,맑음,1℃,30%,건축 / 건축물 / 근린생활시설,"4,892.77㎡","지상 14층, 지하 3층",떨어짐(5미터 이상 ~ 10미터 미만),없음,건축 > 철근콘크리트공사,건설자재 > 철근,설치작업,근린생활시설 / 내부,철근 / 고소,"고소작업 중 추락 위험이 있음에도 불구하고, 안전난간대, 안전고리 착용 등 안전장치...",고소작업 시 추락 위험이 있는 부위에 안전장비 설치.


In [7]:
train['인적사고'].value_counts()

인적사고
물체에 맞음                   3460
끼임                       2549
넘어짐(미끄러짐)                2188
넘어짐(기타)                  2063
기타                       1915
부딪힘                      1815
떨어짐(2미터 미만)              1735
절단, 베임                   1661
넘어짐(물체에 걸림)              1490
떨어짐(2미터 이상 ~ 3미터 미만)      759
떨어짐(분류불능)                 708
떨어짐(3미터 이상 ~ 5미터 미만)      545
깔림                        514
질병                        383
찔림                        306
떨어짐(5미터 이상 ~ 10미터 미만)     298
분류불능                      225
떨어짐(10미터 이상)              210
없음                        196
화상                        181
교통사고                      122
감전                         49
질식                         18
Name: count, dtype: int64

In [14]:
import re

text = """1. **작업 표면 안전 확보**: 모든 작업 플랫폼에 미끄럼 방지 매트 설치 및 정기적인 점검 실시.
2. **안전 교육 강화**: 미끄러짐 위험 인식과 안전한 작업 방법에 대한 정기 교육 실시.
3. **개인 보호 장비 의무화**: 안전화 및 안전벨트 착용을 강제화하며, 특히 미끄러운 환경에서는 추가적인 미끄럼 방지 기능이 있는 장비 사용 권장.
4. **작업 절차 표준화**: 자재 이동 및 해체 작업 시 안전한 절차를 명확히 정의하고 준수를 감독.
5. **정기적 안전 점검**: 해체 작업"""

# 숫자 + 점(.) 형태 제거 (예: "1.", "2." 등)
text = re.sub(r'\d+\.', '', text)

# **내용** 패턴 제거
text = re.sub(r'\*\*.*?\*\*', '', text)

# : 제거
text = re.sub(r':', '', text)

# 마지막 .을 제외한 모든 .을 ,로 변경
text = re.sub(r'\.(?=.*[^.])', ',', text)

# 줄바꿈 제거
text = re.sub(r'\n', ' ', text)

# 2칸 이상 띄어쓰기 제거 (한 칸으로 변환)
text = re.sub(r'\s+', ' ', text)

print(text.strip())


모든 작업 플랫폼에 미끄럼 방지 매트 설치 및 정기적인 점검 실시, 미끄러짐 위험 인식과 안전한 작업 방법에 대한 정기 교육 실시, 안전화 및 안전벨트 착용을 강제화하며, 특히 미끄러운 환경에서는 추가적인 미끄럼 방지 기능이 있는 장비 사용 권장, 자재 이동 및 해체 작업 시 안전한 절차를 명확히 정의하고 준수를 감독, 해체 작업


In [15]:
df = pd.read_csv("C:/Users/minkyu/Desktop/dacon accident prevention/submission-EXAONE-3.5-32B-Instruct.csv")

In [ ]:
df['재발방지대책 및 향후조치계획']

In [18]:
# 정규식을 적용하는 함수 정의
def clean_text(text):
    text = re.sub(r'\d+\.', '', text)          # 숫자 + 점(.) 제거 (예: "1.", "2.")
    text = re.sub(r'\*\*.*?\*\*', '', text)    # **내용** 제거
    text = re.sub(r':', '', text)              # : 제거
    text = re.sub(r'\.(?=.*[^.])', ',', text)  # 마지막 .을 제외한 모든 .을 ,로 변경
    text = re.sub(r'\n', ' ', text)            # 줄바꿈 제거
    text = re.sub(r'\s+', ' ', text)           # 2칸 이상 띄어쓰기 제거 (한 칸으로 변환)
    return text.strip()

# 데이터프레임에 적용
df['재발방지대책 및 향후조치계획'] = df['재발방지대책 및 향후조치계획'].apply(clean_text)

# 결과 확인
print(df)

           ID                                    재발방지대책 및 향후조치계획     vec_0  \
0    TEST_000  지반 안정성 강화를 위해 아웃트리거 설치 전 지반 조사 및 강화 조치 실시, 아웃트... -1.012069   
1    TEST_001  안전 교육 강화와 함께 숫돌 사용 지침 엄격히 준수하도록 교육 실시, 모든 작업자에... -0.566179   
2    TEST_002  안전교육 강화로 주의력 집중 및 주변 환경 인식 훈련 실시, 계단 안전 표시 설치 ... -0.231025   
3    TEST_003  작업 발판 주변 벽돌 잔재 정기적 제거 및 안전 표시 설치, 작업자 안전 교육 강화...  0.589290   
4    TEST_004  작업자에게 점심시간 안전이동 지침 강화 및 정기적인 안전 교육 실시, 안전 경로 표... -1.159659   
..        ...                                                ...       ...   
959  TEST_959  석재 품질 사전 검사 강화, 안전장비 착용 의무화, 파단 위험 석재 사용 제한 및 ...  0.368874   
960  TEST_960  작업 통로 명확히 표시하고 정기적인 안전 점검 실시, 근로자 대상 통로 안전 사용 ... -0.203108   
961  TEST_961  안전 교육 강화 및 개인 보호 장비(PPE) 착용 의무화 실시, 믹서트럭 작업 영역... -0.181741   
962  TEST_962  안전장갑 착용 의무화 및 깨진 자재 처리 전용 보호 장비 제공, 정기적인 안전 교육... -0.113140   
963  TEST_963  방호덮개의 신속한 작동 확인 및 자동화 시스템 도입, 정기적인 장비 점검과 유지보수... -0.295499   

        vec_1     vec_2     vec_3     vec_4     vec_5     vec_6

In [19]:
df['재발방지대책 및 향후조치계획']

0      지반 안정성 강화를 위해 아웃트리거 설치 전 지반 조사 및 강화 조치 실시, 아웃트...
1      안전 교육 강화와 함께 숫돌 사용 지침 엄격히 준수하도록 교육 실시, 모든 작업자에...
2      안전교육 강화로 주의력 집중 및 주변 환경 인식 훈련 실시, 계단 안전 표시 설치 ...
3      작업 발판 주변 벽돌 잔재 정기적 제거 및 안전 표시 설치, 작업자 안전 교육 강화...
4      작업자에게 점심시간 안전이동 지침 강화 및 정기적인 안전 교육 실시, 안전 경로 표...
                             ...                        
959    석재 품질 사전 검사 강화, 안전장비 착용 의무화, 파단 위험 석재 사용 제한 및 ...
960    작업 통로 명확히 표시하고 정기적인 안전 점검 실시, 근로자 대상 통로 안전 사용 ...
961    안전 교육 강화 및 개인 보호 장비(PPE) 착용 의무화 실시, 믹서트럭 작업 영역...
962    안전장갑 착용 의무화 및 깨진 자재 처리 전용 보호 장비 제공, 정기적인 안전 교육...
963    방호덮개의 신속한 작동 확인 및 자동화 시스템 도입, 정기적인 장비 점검과 유지보수...
Name: 재발방지대책 및 향후조치계획, Length: 964, dtype: object

In [20]:
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'
sample = pd.read_csv(data_path+"sample_submission.csv")

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 5. 정제된 문장을 리스트로 변환 후 임베딩
test_results = df['재발방지대책 및 향후조치계획'].tolist()
pred_embeddings = embedding.encode(test_results)  # (샘플 개수, 768)

# 6. submission 파일 로드
submission = pd.read_csv(data_path+"sample_submission.csv", encoding='utf-8-sig')

# 7. submission 파일에 결과 저장
submission.iloc[:, 1] = test_results  # 2번째 컬럼에 정제된 문장 저장
submission.iloc[:, 2:] = pred_embeddings  # 3번째 컬럼부터 임베딩 저장

# 8. CSV 저장



c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
submission.to_csv('submission-EXAONE-3.5-32B-Instruct_정규식적용.csv', index=False, encoding='utf-8-sig')

In [1]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [2]:
data_path = "C:/Users/minkyu/Desktop/open/"
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'

In [3]:
train = pd.read_csv(data_path+"train.csv")
test = pd.read_csv(data_path+"test.csv")
sample = pd.read_csv(data_path+"sample_submission.csv")

In [4]:
grouped = train.groupby("인적사고")

In [5]:
res = {}
cosine_res = []
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    cosine_res += similarity[similarity.mean(axis=1).argmax()].tolist()
    res[name] = plan.iloc[similarity.mean(axis=1).argmax()]

100%|██████████| 23/23 [03:40<00:00,  9.59s/it]


In [6]:
arr = cosine_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

Range 0.0 - 0.1: 1개
Range 0.1 - 0.2: 25개
Range 0.2 - 0.3: 191개
Range 0.3 - 0.4: 665개
Range 0.4 - 0.5: 1657개
Range 0.5 - 0.6: 3794개
Range 0.6 - 0.7: 6597개
Range 0.7 - 0.8: 7557개
Range 0.8 - 0.9: 2783개
Range 0.9 - 1.0: 112개


In [7]:
res

{'감전': '작업자 교육 및 안전장구 착용 철저와 전기작업자 교육 및 피복관리 철저를 통한 재발 방지 대책.',
 '교통사고': '작업업 특별 안전 교육 실시, 신호수 교육 실시, 작업차량 후방 감지 센서 및 카메라 설치, 수시 위험성 평가 실시, 감독관 주체 특별 안전 교육 실시, 현장 점검 시 신호수 위치 점검 등으로 구성된 재발 방지 대책 및 향후 조치 계획.',
 '기타': '작업전 안전교육 및 특별안전교육 실시와 안전관리 및 안전교육 철저를 통한 재발 방지 대책.',
 '깔림': '작업 개시 전 작업내용 숙지 및 안전교육 강화, 수시 현장 방문 및 점검을 통한 재발 방지 대책 강구.',
 '끼임': '작업전 작업자 안전교육 강화, 안전수칙 준수 특별안전교육 실시, 작업지휘자 관리감독 철저에 대한 재발 방지 대책과 향후 조치 계획.',
 '넘어짐(기타)': '작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조치 계획.',
 '넘어짐(물체에 걸림)': '작업전 안전교육 강화 및 작업장 위험요소 점검을 통한 재발 방지와 안전관리 교육 철저를 통한 향후 조치 계획.',
 '넘어짐(미끄러짐)': '이동통로 확보 관리와 작업 전 안전교육 철저 및 정기적 근로자 안전교육 시행을 통한 재발 방지 대책.',
 '떨어짐(10미터 이상)': '안전시설물 설치와 체계적인 안전교육 실시를 통한 재발 방지 대책 및 공사현장 작업중지명령과 안전교육 강화를 포함한 향후 조치 계획.',
 '떨어짐(2미터 미만)': '안전교육 실시와 현장 내 작업지시사항 철저 이행, 안전관리 교육 이행, 안전위험요소 제거 점검 및 대책 강구를 통한 재발 방지 대책.',
 '떨어짐(2미터 이상 ~ 3미터 미만)': '작업 전 안전교육 철저와 안전고리 이중결속 준수, 작업 중 안전시설물 확인 및 사고자 상태 수시 체크를 통한 재발 방지 대책 마련.',
 '떨어짐(3미터 이상 ~ 5미터 미만)': '작업 시 안전교육 및 보호구 착용과 안전매트 설치를 통한 재발 방지 대책

In [8]:
import pandas as pd 
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

# 모델 로드
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

# 결과를 저장할 딕셔너리
category_cosine_res = {}  # 카테고리별 유사도 저장
res = {}  # 대표 문장 저장

# 그룹화 및 처리
grouped = train.groupby("인적사고")
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)
    
    # 평균 유사도가 가장 높은 행의 유사도 값 저장
    max_mean_idx = similarity.mean(axis=1).argmax()
    category_cosine_res[name] = similarity[max_mean_idx].tolist()  # 해당 카테고리의 유사도 리스트
    res[name] = plan.iloc[max_mean_idx]  # 대표 문장 저장

# 범위별 카테고리 분포 계산
bins = np.arange(0, 1.1, 0.1)
category_bin_counts = {name: np.histogram(values, bins=bins)[0] for name, values in category_cosine_res.items()}

# 결과 출력
print("범위별 '인적사고' 카테고리 분포:")
for i in range(len(bins) - 1):
    print(f"\nRange {bins[i]:.1f} - {bins[i+1]:.1f}:")
    for category, counts in category_bin_counts.items():
        if counts[i] > 0:  # 해당 범위에 값이 있는 경우만 출력
            print(f"  {category}: {counts[i]}개")

# 전체 히스토그램 재확인
all_cosine_res = []
for values in category_cosine_res.values():
    all_cosine_res.extend(values)
hist, bin_edges = np.histogram(all_cosine_res, bins=bins)
print("\n전체 히스토그램 확인:")
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(
100%|██████████| 23/23 [03:40<00:00,  9.57s/it]

범위별 '인적사고' 카테고리 분포:

Range 0.0 - 0.1:
  기타: 1개

Range 0.1 - 0.2:
  기타: 2개
  끼임: 3개
  넘어짐(기타): 3개
  넘어짐(물체에 걸림): 3개
  떨어짐(2미터 미만): 3개
  떨어짐(2미터 이상 ~ 3미터 미만): 1개
  물체에 맞음: 8개
  질병: 2개

Range 0.2 - 0.3:
  기타: 18개
  깔림: 8개
  끼임: 35개
  넘어짐(기타): 21개
  넘어짐(물체에 걸림): 9개
  넘어짐(미끄러짐): 12개
  떨어짐(10미터 이상): 2개
  떨어짐(2미터 미만): 6개
  떨어짐(2미터 이상 ~ 3미터 미만): 3개
  떨어짐(5미터 이상 ~ 10미터 미만): 3개
  떨어짐(분류불능): 3개
  물체에 맞음: 31개
  부딪힘: 17개
  분류불능: 6개
  없음: 2개
  절단, 베임: 5개
  질병: 7개
  질식: 1개
  찔림: 2개

Range 0.3 - 0.4:
  감전: 2개
  교통사고: 4개
  기타: 84개
  깔림: 18개
  끼임: 77개
  넘어짐(기타): 63개
  넘어짐(물체에 걸림): 33개
  넘어짐(미끄러짐): 43개
  떨어짐(10미터 이상): 5개
  떨어짐(2미터 미만): 41개
  떨어짐(2미터 이상 ~ 3미터 미만): 9개
  떨어짐(3미터 이상 ~ 5미터 미만): 7개
  떨어짐(5미터 이상 ~ 10미터 미만): 3개
  떨어짐(분류불능): 14개
  물체에 맞음: 126개
  부딪힘: 47개
  분류불능: 8개
  없음: 10개
  절단, 베임: 37개
  질병: 22개
  질식: 2개
  찔림: 8개
  화상: 2개

Range 0.4 - 0.5:
  감전: 8개
  교통사고: 10개
  기타: 205개
  깔림: 49개
  끼임: 188개
  넘어짐(기타): 159개
  넘어짐(물체에 걸림): 82개
  넘어짐(미끄러짐): 120개
  떨어짐(10미터 이상): 13개
  떨어짐(2미터 미만): 76개
  떨어짐(2미터 이

In [9]:
category_bin_counts = {name: np.histogram(values, bins=bins)[0] for name, values in category_cosine_res.items()}

# 결과 출력: 개수와 비율
print("범위별 '인적사고' 카테고리 분포 (개수 및 비율):")
for i in range(len(bins) - 1):
    print(f"\nRange {bins[i]:.1f} - {bins[i+1]:.1f}:")
    for category, counts in category_bin_counts.items():
        total_in_category = sum(counts)  # 해당 카테고리의 전체 유사도 값 개수
        if counts[i] > 0:  # 해당 범위에 값이 있는 경우만 출력
            percentage = (counts[i] / total_in_category) * 100
            print(f"  {category}: {counts[i]}개 ({percentage:.2f}%)")

# 전체 히스토그램 재확인
all_cosine_res = []
for values in category_cosine_res.values():
    all_cosine_res.extend(values)
hist, bin_edges = np.histogram(all_cosine_res, bins=bins)
print("\n전체 히스토그램 확인:")
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

범위별 '인적사고' 카테고리 분포 (개수 및 비율):

Range 0.0 - 0.1:
  기타: 1개 (0.05%)

Range 0.1 - 0.2:
  기타: 2개 (0.10%)
  끼임: 3개 (0.12%)
  넘어짐(기타): 3개 (0.15%)
  넘어짐(물체에 걸림): 3개 (0.20%)
  떨어짐(2미터 미만): 3개 (0.17%)
  떨어짐(2미터 이상 ~ 3미터 미만): 1개 (0.13%)
  물체에 맞음: 8개 (0.23%)
  질병: 2개 (0.52%)

Range 0.2 - 0.3:
  기타: 18개 (0.94%)
  깔림: 8개 (1.56%)
  끼임: 35개 (1.37%)
  넘어짐(기타): 21개 (1.02%)
  넘어짐(물체에 걸림): 9개 (0.60%)
  넘어짐(미끄러짐): 12개 (0.55%)
  떨어짐(10미터 이상): 2개 (0.95%)
  떨어짐(2미터 미만): 6개 (0.35%)
  떨어짐(2미터 이상 ~ 3미터 미만): 3개 (0.40%)
  떨어짐(5미터 이상 ~ 10미터 미만): 3개 (1.01%)
  떨어짐(분류불능): 3개 (0.42%)
  물체에 맞음: 31개 (0.90%)
  부딪힘: 17개 (0.94%)
  분류불능: 6개 (2.67%)
  없음: 2개 (1.02%)
  절단, 베임: 5개 (0.30%)
  질병: 7개 (1.83%)
  질식: 1개 (5.88%)
  찔림: 2개 (0.65%)

Range 0.3 - 0.4:
  감전: 2개 (4.08%)
  교통사고: 4개 (3.28%)
  기타: 84개 (4.39%)
  깔림: 18개 (3.50%)
  끼임: 77개 (3.02%)
  넘어짐(기타): 63개 (3.06%)
  넘어짐(물체에 걸림): 33개 (2.22%)
  넘어짐(미끄러짐): 43개 (1.97%)
  떨어짐(10미터 이상): 5개 (2.38%)
  떨어짐(2미터 미만): 41개 (2.36%)
  떨어짐(2미터 이상 ~ 3미터 미만): 9개 (1.19%)
  떨어짐(3미터 이상 ~ 5미터 미만)

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

# 모델 로드
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

# 결과를 저장할 딕셔너리
category_cosine_res = {}  # 카테고리별 유사도 값 저장
category_sentences = {}   # 카테고리별 문장과 유사도 저장
res = {}                  # 대표 문장 저장

# 그룹화 및 처리
grouped = train.groupby("인적사고")
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)
    
    # 평균 유사도가 가장 높은 행 선택
    max_mean_idx = similarity.mean(axis=1).argmax()
    cosine_values = similarity[max_mean_idx].tolist()  # 해당 행의 유사도 값들
    sentences = plan.tolist()                          # 모든 문장 리스트
    
    # 카테고리별 유사도와 문장 저장
    category_cosine_res[name] = cosine_values
    category_sentences[name] = list(zip(cosine_values, sentences))  # (유사도, 문장) 쌍으로 저장
    res[name] = plan.iloc[max_mean_idx]  # 대표 문장 저장

# 범위 설정
bins = np.arange(0, 1.1, 0.1)

# 특정 범위(0.6~1.0)만 추출
target_ranges = [(0.6, 0.7), (0.7, 0.8), (0.8, 0.9), (0.9, 1.0)]

# 범위별 문장 출력
print("범위별 '인적사고' 카테고리 문장 (최대 10개씩):")
for lower, upper in target_ranges:
    print(f"\nRange {lower:.1f} - {upper:.1f}:")
    for category, sentence_pairs in category_sentences.items():
        # 해당 범위에 속하는 (유사도, 문장) 쌍 필터링
        filtered_sentences = [(sim, sent) for sim, sent in sentence_pairs if lower <= sim < upper]
        if filtered_sentences:
            print(f"  {category}:")
            # 최대 10개 출력
            for sim, sent in filtered_sentences[:10]:
                print(f"    - {sent} (유사도: {sim:.3f})")
            if len(filtered_sentences) > 10:
                print(f"      ... (총 {len(filtered_sentences)}개)")

# 전체 히스토그램 재확인
all_cosine_res = []
for values in category_cosine_res.values():
    all_cosine_res.extend(values)
hist, bin_edges = np.histogram(all_cosine_res, bins=bins)
print("\n전체 히스토그램 확인:")
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(
100%|██████████| 23/23 [03:42<00:00,  9.67s/it]

범위별 '인적사고' 카테고리 문장 (최대 10개씩):

Range 0.6 - 0.7:
  감전:
    - 사고 예방을 위한 면밀한 작업 계획 수립과 안전 관리 강화. (유사도: 0.663)
    - 유체펌프 작동 전 누전 여부 확인 및 주기적 점검과 전기 작업자에게 절연장갑 등 안전관리물품 지급과 우천 시 전기 작업 금지. (유사도: 0.659)
    - 위험 구간 및 장소의 장비 사용 금지와 위험 요소에 대한 사전 안전 조치 강화, 안전 관리자의 교육 및 책임 강화, 작업 근로자의 안전 교육 및 개인 보호구 착용 의무 강화, 전문 인력 배치, 사고 현장에 대한 한시적 즉각 공사 정지 명령과 관련 사항 점검, 해당 과실 관련 주체의 처벌 및 기타 민형사상 행정 처분, 향후 재발 방지 계획 수렴 후 적법성 인정 시 공사 정지 명령 철회. (유사도: 0.601)
    - 안전보완대책 수립과 안전교육 실시, 현장대리인 및 공사감리자에 대한 안전교육과 후속점검 예정. (유사도: 0.657)
    - 향후 작업 시 고압선 보호공 설치와 안전대책 강구 후 현장 작업 중지에 대한 계획. (유사도: 0.602)
    - 전주 이설 후 작업 지시와 안전에 취약한 공종 작업 시 안전관리자 및 공사감독자 확인 후 작업지시의 통합적 관리 방안. (유사도: 0.611)
    - 사고사례 전파 및 안전교육을 통한 재발 방지 대책과 향후 조치 계획. (유사도: 0.618)
    - 안전교육 실시를 통한 재발 방지 대책 및 향후 조치 계획. (유사도: 0.626)
    - 작업주의 및 작업주의 교육을 통한 재발 방지 대책 마련. (유사도: 0.698)
  교통사고:
    - 덤프작업구간 지반 정리 및 사전검토 철저, 신호수 교육 철저와 재발방지대책계획 수립을 위한 상세 조사에 따른 이행 철저 및 신호수 장비운전수 교육 철저. (유사도: 0.659)
    - 안전사고 재발 방지와 예방을 위한 대책 및 조치계획 수립, 안전사고 예방 교육 철저, 안전시설물 추가 설치 등. 

In [11]:
# 텍스트 파일로 저장
with open('output.txt', 'w', encoding='utf-8') as f:
    # 범위별 문장 출력
    f.write("범위별 '인적사고' 카테고리 문장 (최대 10개씩):\n")
    for lower, upper in target_ranges:
        f.write(f"\nRange {lower:.1f} - {upper:.1f}:\n")
        for category, sentence_pairs in category_sentences.items():
            # 해당 범위에 속하는 (유사도, 문장) 쌍 필터링
            filtered_sentences = [(sim, sent) for sim, sent in sentence_pairs if lower <= sim < upper]
            if filtered_sentences:
                f.write(f"  {category}:\n")
                # 최대 10개 출력
                for sim, sent in filtered_sentences[:10]:
                    f.write(f"    - {sent} (유사도: {sim:.3f})\n")
                if len(filtered_sentences) > 10:
                    f.write(f"      ... (총 {len(filtered_sentences)}개)\n")

    # 전체 히스토그램 재확인
    all_cosine_res = []
    for values in category_cosine_res.values():
        all_cosine_res.extend(values)
    hist, bin_edges = np.histogram(all_cosine_res, bins=bins)
    f.write("\n전체 히스토그램 확인:\n")
    for i in range(len(hist)):
        f.write(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개\n")

print("결과가 'output.txt' 파일에 저장되었습니다.")

결과가 'output.txt' 파일에 저장되었습니다.


In [12]:
import numpy as np
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [21]:
# 문장 예시
preds = [
    "작업 전 작업자 안전교육 및 안전관리 철저에 대한 재발 방지 대책과 향후 조치 계획",
]

gts = [
    "위험성 평가 및 교육을 통해 작업장 내 위험요인과 안전수칙을 근로자에게 전파하고, 근로자 안전교육을 강화하며, 본 사고와 관련된 유사 피해 발생 방지를 위한 안전교육을 실시할 계획.",
]

In [19]:
# 샘플에 대한 Cosine Similarity 산식
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [22]:
sample_scores = []
for pred, gt in zip(preds, gts):
    # 생성된 답변 내용을 768 Embedding Vector로 변환
    pred_embed = model.encode(pred)
    gt_embed = model.encode(gt)
    
    sample_score = cosine_similarity(gt_embed, pred_embed)
    # Cosine Similarity Score가 0보다 작으면 0으로 간주
    sample_score = max(sample_score, 0)
    print('예측 : ', pred)
    print('정답 : ', gt)
    print('Cosine Similarity Score : ', sample_score)
    print('-'*20)
    sample_scores.append(sample_score)
print('전체 샘플의 Cosine Similarity Score 평균 : ', np.mean(sample_scores))

예측 :  작업 전 작업자 안전교육 및 안전관리 철저에 대한 재발 방지 대책과 향후 조치 계획
정답 :  위험성 평가 및 교육을 통해 작업장 내 위험요인과 안전수칙을 근로자에게 전파하고, 근로자 안전교육을 강화하며, 본 사고와 관련된 유사 피해 발생 방지를 위한 안전교육을 실시할 계획.
Cosine Similarity Score :  0.75336784
--------------------
전체 샘플의 Cosine Similarity Score 평균 :  0.75336784
